In [29]:
from lxml import html
import requests
import pandas as pd 
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import subprocess as sp
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os
import argparse
import subprocess

def fill_and_send (df):
    df_done_list=[]
    for index, row in df.iterrows() :
        a=get_the_cover_pdf(row['Institute name'],row['Institute adress'])
        subject=('Bewerbung für die stelle "Studentische hilfskraft" von der '+row['Institute name'])
        body =  '''\
        <html>
          <head></head>
          <body>
            <p>Sehr geehrte Herr '''+row['Name_mod']+''', <br><br>
               Im anhang finden Sie meine Bewerbungsunterlagen für die oben genannte Stelle sowie meine Notenspiegel<br>
               <p style="text-align: right;">Ich freue mich sehr auf Ihren Rückmeldung</p><br>
               <p style="text-align: right;"><sub><u>Generated by:Daryl the Bewerbung Slave ver 1.0</u></sub></p>
            </p>
          </body>
        </html>
        '''
        initial_repertory={'Found keyword':row['Found keyword'],'Name':row['Name'],'Mail':row['Mail'],'Institute adress':row['Institute adress'],'Institute name':row['Institute name'],'Link':row['Link']}
        try :
            send_mail(row['Mail'],subject,body,a)
            df_done_list.append(initial_repertory)
        except : 
            pass
    df_done=pd.DataFrame(df_done_list)
    return df_done


#####################latex python interaction function and pdf generation##########

def generate_cover_letter():
    os.chdir("C:\\Users\\hp\\Desktop\\ndhif\\test\\data\\")
    os.system("lualatex --interaction=nonstopmode coverletter.tex")


def get_the_cover_pdf(institute,adress):
    
    content = r'''\usepackage[utf8]{inputenc}
    \newcommand{\institute}{%(INSTITUTE)s}
    \newcommand{\adressie}{%(ADRESS)s}'''
    args=dict()
    args['INSTITUTE']=institute
    args['ADRESS']=adress
    with open('var.tex','w',encoding='utf8') as f:
        f.write(content%args)
    cmd = ['lualatex', '-interaction', 'nonstopmode', 'var.tex']
    proc = subprocess.Popen(cmd)
    proc.communicate()
    generate_cover_letter()
    pdf_path=['coverletter.pdf','cv.pdf']
    return pdf_path

############mail sender function##
def send_mail(receiver,subject,body,file_list):

    msg=MIMEMultipart('alternative')
    sender = 'khalil.jlif@rwth-aachen.de'
    msg['From']=sender
    msg['To']=receiver
    msg['Subject']=subject
    
    
    msg.attach(MIMEText(body,'html'))

    for filename in file_list:

        attachement=open(filename,'rb')
        part=MIMEBase('application','octet-stream')
        part.set_payload(attachement.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition',"attachement; filename= "+ filename)
        msg.attach(part)

        text=msg.as_string()
    server=smtplib.SMTP('mail.rwth-aachen.de',587 )
    server.starttls()

    server.login('nm741271@rwth-aachen.de','********')
    server.sendmail(sender,receiver,text)
    server.quit()

########data_frame_generator_functions#########
    
def get_rwth_root(number_pages):
    roots=[]
    page=1
    
    while (page<=number_pages):
        print("page",page,"from",number_pages)
        rwth_url=requests.get('https://www.rwth-aachen.de/cms/root/Die-RWTH/Arbeiten-an-der-RWTH/~buym/Jobboerse/?page='+str(page))
        rwth_text=rwth_url.content
        mensa_soup=BeautifulSoup(rwth_text)
        link_in_page = []
        for a in mensa_soup.findAll("a",{'target':'_self'}):
            root =(a.get('href')) 
            if 'Einzelansicht'in root:
                link_in_page.append(root)
                table=(a.string).upper()
                if ("STUDENTISCHE HILFSKRAFT" in table):
                    roots.append(root)
                    
        if len(link_in_page) == 0:
                break
        page=page+1
    return roots


def check_keywords(stringewi , key_words):
        keywords_list=[] 
        big_string=stringewi.upper()
        for i in key_words :
            if i in big_string : 
                keywords_list.append(i)
        return keywords_list

def name_trim (stringie) : 
    aaa=stringie
    to_trim=to_trim=['M. Sc.','Dipl.-Wirt.-Ing.','M.Sc.','Dipl.-Phys. ',' M.Sc',' MSc']
    for i in to_trim:
        if i in stringie :
            aaa=stringie.replace(i,'')
            if 'RWTH' in aaa :
                aaa=aaa.replace('RWTH','')
    return aaa
#key_words=['PYTHON','MATLAB','C++','C#','JAVA','ELEKTROTECH','INFORMATIK','REGELUNG','WEB','MODELLIER','MESSTECH','DATA','DIGITALIS']        

def rwth_bewerbung_data(a,key_words):
    
    my_roots=get_rwth_root(a)
    repertory_list=[]

    for i in range (len(my_roots)):
        
        print("loading",int(i/len(my_roots)*100),"%")
        my_root_string=(my_roots[i])
        rwth_url_jobs=requests.get(my_root_string)
        rwth_url_text=rwth_url_jobs.content
        rwth_url_text_path=html.fromstring(rwth_url_text)
        soup=BeautifulSoup(rwth_url_text)

        for a in soup.findAll("div",{"class":"text"}):
            
            to_search=str(a.findAll('p'))
            found_keywords=check_keywords(to_search,key_words)
            
            if (len(found_keywords)==0):
                break
            #subject_name=a.find('h2').contents[0]
            betreuer_name=a.find('p',{'class':'fn n'}).string
            betreuer_mailie=rwth_url_text_path.xpath('//div[@id="main"]//dd[@class="value"]//a/@href')
            betreuer_mailiee=''.join(betreuer_mailie)
            betreuer_mail=betreuer_mailiee.replace('mailto:','')
            adress_xpath='//div[@id="main"]//table[@style="width: 100%;"]//tbody//tr[3]//td[2]/text()'
            adress=rwth_url_text_path.xpath(adress_xpath)

            if len(adress)==0 or len(adress)==1 : 
                
                adress_xpath='//div[@id="main"]//table[@style="width: 100%;"]//tbody//tr[2]//td[2]/text()'
                adress=rwth_url_text_path.xpath(adress_xpath)

            institut_name=rwth_url_text_path.xpath('//div[@id="main"]//div[@class="text"]//p[1]/text()')[-1]
            initial_repertory={'Found keyword':' '.join(found_keywords),'Name':betreuer_name,'Mail':betreuer_mail,'Institute adress':''.join(adress),'Institute name':institut_name,'Link':my_root_string}
            repertory_list.append(initial_repertory)
    
    bewerbung_infos=pd.DataFrame(repertory_list)
    bewerbung_infos['Name_mod'] = bewerbung_infos.apply(lambda x:name_trim(x['Name']),axis=1)
    return bewerbung_infos

key_words=['PYTHON','MATLAB','C++','C#','JAVA','ELEKTROTECH','INFORMATIK','REGELUNG','WEB','MODELLIER','MESSTECH','DATA','DIGITALIS']
fill_and_send(rwth_bewerbung_data(3,key_words))
